In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [2]:
file_path = 'balanced_vehicle_collisions_sample.csv'
try:
    df = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"Error: '{file_path}' not found. Please upload the file.")
    exit()
df

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,01/08/2023,22:37,BRONX,10453.0,40.852703,-73.921844,"(40.852703, -73.921844)",RICHMAN PLAZA,HARLEM RIVER PARK BRIDGE,NaN,...,NaN,NaN,NaN,NaN,4596422,Sedan,NaN,NaN,NaN,NaN
1,05/26/2014,14:54,BRONX,10458.0,40.860197,-73.897710,"(40.8601968, -73.8977096)",EAST 187 STREET,RYER AVENUE,NaN,...,NaN,NaN,NaN,NaN,341239,PASSENGER VEHICLE,NaN,NaN,NaN,NaN
2,08/23/2015,0:15,BRONX,10451.0,40.826398,-73.921154,"(40.8263977, -73.9211541)",EAST 161 STREET,SHERIDAN AVENUE,NaN,...,NaN,NaN,NaN,NaN,3282746,PASSENGER VEHICLE,NaN,NaN,NaN,NaN
3,01/17/2015,21:45,BRONX,10473.0,40.825773,-73.852763,"(40.8257734, -73.8527628)",QUIMBY AVENUE,OLMSTEAD AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,3156068,SPORT UTILITY / STATION WAGON,BICYCLE,NaN,NaN,NaN
4,05/09/2023,16:50,BRONX,10469.0,40.859180,-73.839066,"(40.85918, -73.839066)",KINGSLAND AVENUE,ASTOR AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4627492,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26319,08/24/2021,15:00,Unknown,NaN,40.606743,-74.130810,"(40.606743, -74.13081)",NaN,NaN,211 WHEELER AVENUE,...,Unspecified,NaN,NaN,NaN,4451958,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
26320,06/18/2017,3:15,Unknown,NaN,40.685140,-73.820600,"(40.68514, -73.8206)",107 AVENUE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,3693977,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN
26321,05/19/2018,16:00,Unknown,NaN,40.796600,-73.970390,"(40.7966, -73.97039)",WEST 99 STREET,NaN,NaN,...,NaN,NaN,NaN,NaN,3904284,Taxi,NaN,NaN,NaN,NaN
26322,08/03/2017,6:20,Unknown,NaN,40.748730,-73.937910,"(40.74873, -73.93791)",JACKSON AVENUE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,3724309,Tractor Truck Diesel,Sedan,NaN,NaN,NaN


In [3]:
df['CRASH_DT'] = pd.to_datetime(df['CRASH DATE'] + ' ' + df['CRASH TIME'], errors='coerce')
df = df.dropna(subset=['CRASH_DT'])
df['CRASH_YEAR'] = df['CRASH_DT'].dt.year

In [6]:
df_geo = df.dropna(subset=['LATITUDE', 'LONGITUDE']).copy()
df_geo = df_geo[(df_geo['LATITUDE'] != 0) & (df_geo['LONGITUDE'] != 0)]

In [9]:
df['Hour'] = df['CRASH_DT'].dt.hour
df['DayOfWeek'] = df['CRASH_DT'].dt.dayofweek
df['Month'] = df['CRASH_DT'].dt.month
df['Is_Weekend'] = df['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)
df['Rush_Hour'] = df['Hour'].apply(lambda x: 1 if (7 <= x <= 9) or (16 <= x <= 19) else 0)
df['BOROUGH'] = df['BOROUGH'].fillna('Unknown')
df['VEHICLE TYPE CODE 1'] = df['VEHICLE TYPE CODE 1'].fillna('Unspecified')
df['CONTRIBUTING FACTOR VEHICLE 1'] = df['CONTRIBUTING FACTOR VEHICLE 1'].fillna('Unspecified')
imputer = SimpleImputer(strategy='mean')
df['NUMBER OF PERSONS INJURED'] = imputer.fit_transform(df[['NUMBER OF PERSONS INJURED']])
df['NUMBER OF PERSONS KILLED'] = imputer.fit_transform(df[['NUMBER OF PERSONS KILLED']])


In [10]:
def group_rare_categories(series, n_top=20):
    top_n = series.value_counts().nlargest(n_top).index
    return series.apply(lambda x: x if x in top_n else 'Other')

df['VEHICLE_GROUP'] = group_rare_categories(df['VEHICLE TYPE CODE 1'])
df['FACTOR_GROUP'] = group_rare_categories(df['CONTRIBUTING FACTOR VEHICLE 1'])

In [11]:
le_borough = LabelEncoder()
df['BOROUGH_Code'] = le_borough.fit_transform(df['BOROUGH'])

le_vehicle = LabelEncoder()
df['VEHICLE_Code'] = le_vehicle.fit_transform(df['VEHICLE_GROUP'])

le_factor = LabelEncoder()
df['FACTOR_Code'] = le_factor.fit_transform(df['FACTOR_GROUP'])

In [12]:
features = ['Hour', 'DayOfWeek', 'Month', 'Is_Weekend', 'Rush_Hour', 'BOROUGH_Code', 'VEHICLE_Code']
X = df[features]

In [13]:
print(f"Data ready. Total records: {len(df)}")

Data ready. Total records: 26324


In [14]:
print("\n--- Training Model 1: Severity Prediction ---")


--- Training Model 1: Severity Prediction ---


In [15]:
df['IS_SEVERE'] = ((df['NUMBER OF PERSONS INJURED'] > 0) | (df['NUMBER OF PERSONS KILLED'] > 0)).astype(int)
y_sev = df['IS_SEVERE']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y_sev, test_size=0.2, random_state=42)


In [17]:
clf_severity = RandomForestClassifier(n_estimators=100, max_depth=15, min_samples_leaf=5, class_weight='balanced', random_state=42)
clf_severity.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=15,
                       min_samples_leaf=5, random_state=42)

In [18]:
y_pred_sev = clf_severity.predict(X_test)
print("Severity Prediction Accuracy:", accuracy_score(y_test, y_pred_sev))
print(classification_report(y_test, y_pred_sev))

Severity Prediction Accuracy: 0.6537511870845204
              precision    recall  f1-score   support

           0       0.78      0.75      0.77      3998
           1       0.30      0.34      0.32      1267

    accuracy                           0.65      5265
   macro avg       0.54      0.55      0.55      5265
weighted avg       0.67      0.65      0.66      5265



In [19]:
print("\n--- Model 2: Injury Count Prediction ---")
y_inj = df['NUMBER OF PERSONS INJURED']
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X, y_inj, test_size=0.2, random_state=42)


--- Model 2: Injury Count Prediction ---


In [20]:
reg_injured = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_leaf=10,
    random_state=42
)
reg_injured.fit(X_train_r, y_train_r)

RandomForestRegressor(max_depth=10, min_samples_leaf=10, random_state=42)

In [21]:
y_pred_r = reg_injured.predict(X_test_r)
print(f"RMSE (Root Mean Squared Error): {np.sqrt(mean_squared_error(y_test_r, y_pred_r)):.4f}")

RMSE (Root Mean Squared Error): 0.6868


In [22]:
print("\n--- Training Model 3: Primary Cause Prediction ---")


--- Training Model 3: Primary Cause Prediction ---


In [23]:
y_cause = df['FACTOR_Code']
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X, y_cause, test_size=0.2, random_state=42)

In [24]:
clf_cause = RandomForestClassifier(n_estimators=100, max_depth=15, min_samples_leaf=5, class_weight='balanced', random_state=42)
clf_cause.fit(X_train_c, y_train_c)

RandomForestClassifier(class_weight='balanced', max_depth=15,
                       min_samples_leaf=5, random_state=42)

In [25]:
print(f"Accuracy on Major Causes: {clf_cause.score(X_test_c, y_test_c):.2%}")

Accuracy on Major Causes: 9.61%


In [26]:
y_pred_cause = clf_cause.predict(X_test_c)
print("Primary Cause Prediction Accuracy:", accuracy_score(y_test_c, y_pred_cause))
print(classification_report(y_test_c, y_pred_cause, target_names=le_factor.classes_))

Primary Cause Prediction Accuracy: 0.09610636277302943
                                precision    recall  f1-score   support

           Alcohol Involvement       0.09      0.32      0.14        66
              Backing Unsafely       0.04      0.05      0.04       187
Driver Inattention/Distraction       0.14      0.01      0.02      1040
           Driver Inexperience       0.01      0.01      0.01        87
 Failure to Yield Right-of-Way       0.08      0.05      0.06       332
               Fatigued/Drowsy       0.09      0.38      0.14       108
         Following Too Closely       0.13      0.15      0.14       293
            Lost Consciousness       0.02      0.12      0.04        49
                         Other       0.10      0.06      0.08       276
               Other Vehicular       0.05      0.06      0.05       180
           Passing Too Closely       0.05      0.16      0.07       125
Passing or Lane Usage Improper       0.04      0.09      0.05       128
        

In [27]:
print("\n--- Training Model 4: Hotspot Clustering ---")


--- Training Model 4: Hotspot Clustering ---


In [28]:
if len(df_geo) > 0:
    kmeans = KMeans(n_clusters=5, n_init=10, random_state=42)
    kmeans.fit(df_geo[['LATITUDE', 'LONGITUDE']])
    df_geo['Cluster'] = kmeans.labels_
    print(f"Cluster Centers identified: {len(kmeans.cluster_centers_)}")
else:
    print("Warning: Not enough valid geospatial data for clustering.")

Cluster Centers identified: 5


In [29]:
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='Hour', bins=24, kde=True, color='skyblue')
plt.title('Figure 4.1: Histogram of Accidents by Hour of Day')
plt.xlabel('Hour of Day (0-23)')
plt.ylabel('Number of Accidents')
plt.grid(axis='y', alpha=0.5)
plt.savefig('Figure_4.1_Histogram_Hour.png')
plt.close()

In [30]:
plt.figure(figsize=(10, 8))
corr_matrix = df[features + ['IS_SEVERE', 'NUMBER OF PERSONS INJURED']].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Figure 4.2: Correlation Heatmap of Features')
plt.savefig('Figure_4.2_Correlation_Heatmap.png')
plt.close()

In [31]:
plt.figure(figsize=(12, 6))
top_factors = df['CONTRIBUTING FACTOR VEHICLE 1'].value_counts().head(10)
sns.barplot(x=top_factors.values, y=top_factors.index, palette='viridis')
plt.title('Figure 4.3: Top 10 Contributing Factors for Accidents')
plt.xlabel('Count')
plt.savefig('Figure_4.3_Top_Factors.png')
plt.close()

C:\Users\priya\AppData\Local\Temp\ipykernel_3196\4208626361.py:3: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=top_factors.values, y=top_factors.index, palette='viridis')


In [32]:
plt.figure(figsize=(10, 8))
sns.scatterplot(data=df_geo, x='LONGITUDE', y='LATITUDE', hue='Cluster', palette='viridis', s=10, alpha=0.6)
plt.title('Accident Hotspots (K-Means Clusters)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.xlim(df_geo['LONGITUDE'].quantile(0.01), df_geo['LONGITUDE'].quantile(0.99))
plt.ylim(df_geo['LATITUDE'].quantile(0.01), df_geo['LATITUDE'].quantile(0.99))
plt.legend(title='Cluster')
plt.savefig('hotspots_map.png')
plt.close()

In [99]:
plt.figure(figsize=(10, 6))
importances = clf_severity.feature_importances_
indices = np.argsort(importances)[::-1]
plt.title('Feature Importance (Severity Prediction)')
plt.bar(range(X.shape[1]), importances[indices], align='center')
plt.xticks(range(X.shape[1]), [features[i] for i in indices], rotation=45)
plt.tight_layout()
plt.savefig('feature_importance.png')
plt.close()

In [93]:
y_pred = clf_severity.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Minor', 'Severe'], yticklabels=['Minor', 'Severe'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix: Severity Prediction')
plt.savefig('confusion_matrix.png')
plt.close()

In [96]:
y_inj = df['NUMBER OF PERSONS INJURED'].fillna(0)
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X, y_inj, test_size=0.2, random_state=42)
reg = RandomForestRegressor(n_estimators=50, random_state=42)
reg.fit(X_train_r, y_train_r)
y_pred_r = reg.predict(X_test_r)

plt.figure(figsize=(10, 6))
plt.scatter(y_test_r, y_pred_r, alpha=0.3)
plt.plot([0, y_test_r.max()], [0, y_test_r.max()], 'r--', lw=2) # Identity line
plt.xlabel('Actual Injuries')
plt.ylabel('Predicted Injuries')
plt.title('Actual vs. Predicted Injuries (Regression)')
plt.savefig('regression_actual_vs_pred.png')
plt.close()